# torch.nn.parameter.Buffer

- **Buffer** нь PyTorch-ийн параметр биш ч гэсэн сүлжээний хэсэг болох тогтмол өгөгдлийг хадгалахад зориулагдсан. Энэ нь суралцагдах параметрүүдээс (nn.Parameter) ялгаатай нь градиент тооцоонд ордоггүй, буцаан суралцагдахгүй утгууд юм.

## Суурь хэрэглээ:

In [1]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        
        # Суралцагдах параметр
        self.weight = nn.Parameter(torch.randn(hidden_size, input_size))
        
        # Buffer (суралцагдахгүй тогтмол утга)
        self.register_buffer('running_mean', torch.zeros(hidden_size))
        self.register_buffer('running_var', torch.ones(hidden_size))
        
        # persistent=False үед state_dict-д хадгалагдахгүй
        self.register_buffer('temp_buffer', torch.tensor([1.0, 2.0, 3.0]), persistent=False)
    
    def forward(self, x):
        # Buffer ашиглах
        normalized = (x - self.running_mean) / torch.sqrt(self.running_var + 1e-5)
        return torch.matmul(normalized, self.weight.t())

In [14]:
model = MyModel(input_size=10, hidden_size=5)
for name, param in model.named_parameters():
    print(f"Parameter - Name: {name}, Shape: {param.shape}")
print(model.running_mean)    
print(model.running_var)
print(model.state_dict())

Parameter - Name: weight, Shape: torch.Size([5, 10])
tensor([0., 0., 0., 0., 0.])
tensor([1., 1., 1., 1., 1.])
OrderedDict([('weight', tensor([[-0.3374,  0.4554,  0.4736, -0.6821,  0.8839,  2.9157, -0.7849,  0.2084,
          1.1419,  2.0074],
        [-2.2716, -2.0676,  1.7437,  1.8423,  1.0641, -1.8200, -2.3000,  0.1482,
          0.5342, -0.4623],
        [-0.1153,  0.6981,  1.2750,  0.6758, -0.6575, -0.4092,  0.0484,  0.9167,
          0.1040,  0.8143],
        [ 0.0188,  0.8150,  1.5313, -0.4581,  1.3372,  0.8090, -0.7218,  0.2599,
         -0.5052,  1.2041],
        [ 1.3950,  0.4390, -1.4032,  1.4993,  1.4297, -0.4141,  0.1363,  0.1492,
          0.7302, -0.1614]])), ('running_mean', tensor([0., 0., 0., 0., 0.])), ('running_var', tensor([1., 1., 1., 1., 1.]))])


## Тодорхойлолт:

In [9]:
torch.nn.parameter.Buffer(data=None, persistent=True)

tensor([])

- **data** (Tensor): Buffer-т хадгалагдах тензор

- **persistent** (bool): True үед state_dict-д хадгалагдана. False үед хадгалагдахгүй

## Жишээнүүд:

### 1. Үндсэн Buffer бүртгэх

In [10]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Buffer бүртгэх арга 1: register_buffer ашиглах
        self.register_buffer('mean', torch.tensor([0.0]))
        self.register_buffer('std', torch.tensor([1.0]))
        
        # Buffer бүртгэх арга 2: Шууд үүсгэх
        self.count = torch.tensor(0)  # Энэ нь buffer болохгүй!
        
    def forward(self, x):
        # Зөвхөн бүртгэгдсэн buffer-ууд л зөв ажиллана
        return (x - self.mean) / self.std

### 2. BatchNorm дэх buffer-ын жишээ

In [11]:
class CustomBatchNorm(nn.Module):
    def __init__(self, num_features, momentum=0.1):
        super().__init__()
        self.num_features = num_features
        self.momentum = momentum
        
        # Суралцагдах параметрүүд
        self.weight = nn.Parameter(torch.ones(num_features))
        self.bias = nn.Parameter(torch.zeros(num_features))
        
        # Суралцагдахгүй buffer-ууд
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))
        self.register_buffer('num_batches_tracked', torch.tensor(0, dtype=torch.long))
    
    def forward(self, x):
        if self.training:
            # Сургалтын горимд статистик шинэчлэх
            mean = x.mean(dim=0)
            var = x.var(dim=0, unbiased=False)
            
            # Exponential moving average
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var
            self.num_batches_tracked += 1
            
            # Normalize
            x_normalized = (x - mean) / torch.sqrt(var + 1e-5)
        else:
            # Шалгалтын горимд хадгалсан статистик ашиглах
            x_normalized = (x - self.running_mean) / torch.sqrt(self.running_var + 1e-5)
        
        return self.weight * x_normalized + self.bias

### 3. Positional Encoding buffer

In [12]:
class TransformerModel(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        self.d_model = d_model
        
        # Positional encoding-ийг buffer хэлбэрээр хадгалах
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           (-torch.log(torch.tensor(10000.0)) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)  # Нэг удаа тооцоолж хадгалах
    
    def forward(self, x):
        # Positional encoding нэмэх
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len]

### 4. Persistent vs Non-persistent

In [13]:
class BufferExample(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Persistent buffer (state_dict-д хадгалагдана)
        self.register_buffer('important_stats', 
                           torch.tensor([1.0, 2.0, 3.0]), 
                           persistent=True)
        
        # Non-persistent buffer (state_dict-д хадгалагдахгүй)
        self.register_buffer('temp_cache', 
                           torch.zeros(10, 10), 
                           persistent=False)
        
        # persistent параметрийн анхны утга: True
        self.register_buffer('default_buffer', torch.ones(5))
    
    def state_dict_example(self):
        # state_dict-ыг харах
        state = self.state_dict()
        print("State dict keys:", list(state.keys()))
        # Гаралт: ['important_stats', 'default_buffer']
        # 'temp_cache' хамаарахгүй!

## Бусад хэрэглээ:
### 1. Маск хадгалах

In [15]:
class MaskedLayer(nn.Module):
    def __init__(self, size):
        super().__init__()
        
        # Маскийг buffer хэлбэрээр хадгалах
        mask = torch.tril(torch.ones(size, size))
        self.register_buffer('mask', mask)
        
        self.weight = nn.Parameter(torch.randn(size, size))
    
    def forward(self, x):
        # Маск ашиглан жинг хязгаарлах
        masked_weight = self.weight * self.mask
        return torch.matmul(x, masked_weight)

### 2. Look-up хүснэгт

In [16]:
class LookupTable(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        
        # Embedding параметр
        self.embedding = nn.Parameter(torch.randn(vocab_size, embedding_dim))
        
        # Precomputed frequency table (buffer)
        freq = torch.ones(vocab_size) / vocab_size
        self.register_buffer('frequency', freq)
        
        # Positional index (buffer)
        positions = torch.arange(vocab_size)
        self.register_buffer('pos_indices', positions)
    
    def forward(self, token_ids):
        embeddings = self.embedding[token_ids]
        weighted = embeddings * self.frequency[token_ids].unsqueeze(-1)
        return weighted

## Чухал шинж чанарууд:
### Buffer-ын онцлог:

In [17]:
model = nn.Module()

# Buffer үүсгэх
model.register_buffer('my_buffer', torch.tensor([1.0, 2.0, 3.0]))

# Шинж чанарууд:
print("Requires grad:", model.my_buffer.requires_grad)  # False
print("Is parameter:", isinstance(model.my_buffer, nn.Parameter))  # False
print("In parameters():", 'my_buffer' in dict(model.named_parameters()))  # False
print("In buffers():", 'my_buffer' in dict(model.named_buffers()))  # True

Requires grad: False
Is parameter: False
In parameters(): False
In buffers(): True


### Buffer-тай ажиллах:

In [18]:
class BufferOperations(nn.Module):
    def __init__(self):
        super().__init__()
        self.register_buffer('stats', torch.zeros(3))
    
    def update_buffer(self, new_data):
        # Buffer шинэчлэх (градиент шаардлагагүй)
        with torch.no_grad():
            self.stats += new_data
    
    def reset_buffer(self):
        # Buffer дахин эхлүүлэх
        self.stats.zero_()
    
    def use_buffer_in_computation(self, x):
        # Buffer тооцоололд ашиглах
        # Градиент buffer-аас үүсэхгүй
        return x * self.stats.unsqueeze(0)

## Анхаарах зүйлс:

- Градиент: Buffer-ууд requires_grad=False байна

- Хөдөлгөөн: to(device), cpu(), cuda() зэрэг үйлдлүүд buffer-уудад нөлөөлнө

- Хадгалалт: torch.save()/torch.load() үйлдлүүд persistent buffer-уудыг хадгална

- Клончлох: copy.deepcopy() нь buffer-уудыг зөв хуулах болно

- Төрөл: Buffer нь тензорын төрөл, device-тэй нийцэх ёстой

## Практик зөвлөмж:

1. Батч нормалчлал: Статистик мэдээллийг buffer-д хадгал

2. Кэш: Давтан тооцоолол хийхгүйгээр үр дүнг хадгал

3. Маск: Градиент шаардлагагүй тогтмол матрицууд

4. Тохиргоо: Моделийн тохиргоог хадгалах

5. Тоолуур: Давталт, алхамыг тоолох

Buffer нь PyTorch моделд тогтмол, суралцагдахгүй өгөгдлийг үр дүнтэй хадгалахад чухал үүрэгтэй.